In [1]:
import numpy as np
import pandas as pd
import string
import random
import math
import nltk
import nltk
from nltk import pos_tag, NaiveBayesClassifier, FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [2]:
df=pd.read_csv("tweets.csv")

In [3]:
df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24-02-2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,24-02-2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,5.695880e+17,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,22-02-2015 12:01,NaN,NaN
14636,5.695870e+17,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,22-02-2015 11:59,Texas,NaN
14637,5.695870e+17,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,22-02-2015 11:59,"Nigeria,lagos",NaN
14638,5.695870e+17,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,22-02-2015 11:59,New Jersey,Eastern Time (US & Canada)


In [4]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24-02-2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,24-02-2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)


In [5]:
df.shape

(14640, 15)

In [6]:
df.info

<bound method DataFrame.info of            tweet_id airline_sentiment  airline_sentiment_confidence  \
0      5.703060e+17           neutral                        1.0000   
1      5.703010e+17          positive                        0.3486   
2      5.703010e+17           neutral                        0.6837   
3      5.703010e+17          negative                        1.0000   
4      5.703010e+17          negative                        1.0000   
...             ...               ...                           ...   
14635  5.695880e+17          positive                        0.3487   
14636  5.695870e+17          negative                        1.0000   
14637  5.695870e+17           neutral                        1.0000   
14638  5.695870e+17          negative                        1.0000   
14639  5.695870e+17           neutral                        0.6771   

               negativereason  negativereason_confidence         airline  \
0                         NaN          

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  float64
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [8]:
texts=df["text"].values
categories=df["airline_sentiment"].values

In [9]:
texts=[word_tokenize(text) for text in texts]

In [10]:
documents=[(texts[i], categories[i]) for i in range(len(texts))]

In [11]:
stop_words=stopwords.words("english")

punctuation_marks=string.punctuation

numbers=np.arange(0, 10)

In [12]:
punctuation_marks=[punctuation_mark for punctuation_mark in punctuation_marks]

In [13]:
stop_words=np.concatenate((stop_words, punctuation_marks, numbers), axis=0)

In [14]:
lemmatizer=WordNetLemmatizer()

In [15]:
ADJ, ADV, VERB, NOUN=('a', 's', 'v', 'n')

In [16]:
def get_simple_pos_tag(tag) :
    if(tag.startswith('J')) :
        return ADJ
    elif(tag.startswith('V')) :
        return VERB
    elif(tag.startswith('R')) :
        return ADV
    else :
        return NOUN

In [17]:
def clean_words(words) :
    cleaned_words=[]

    pos_tag_values=pos_tag(words)

    for i in range(len(words)) :
        word=words[i]

        if(word.lower() not in stop_words) :
            number_present=False

            for char in word :
                if(char.isnumeric()) :
                    number_present=True

                    break

            if(number_present) :
                continue
            
            root_word=lemmatizer.lemmatize(word, pos=get_simple_pos_tag(pos_tag_values[i][1]))

            cleaned_words.append(root_word)

    return cleaned_words

In [18]:
cleaned_data=[]

for (words, category) in documents :
    cleaned_data.append((clean_words(words), category))

In [19]:
random.shuffle(cleaned_data)

In [20]:
total=len(cleaned_data)

limit_75=math.floor(total*0.75)

In [21]:
train_data=cleaned_data[: limit_75]
test_data=cleaned_data[limit_75:]

In [22]:
all_words=[]

for document in train_data :
    all_words+=document[0]

In [23]:
words_freqs=FreqDist(all_words)

top_words=words_freqs.most_common(2000)

In [24]:
features=[word[0] for word in top_words]

In [25]:
def get_feature_dictionary(words) :
    feature_dict={}

    words_set=set(words) 

    for word in features :
        feature_dict[word]=(word in words_set)

    return feature_dict

In [26]:
train_data=[(get_feature_dictionary(document), category) for (document, category) in train_data]
test_data=[(get_feature_dictionary(document), category) for (document, category) in test_data]

In [27]:
classifier=NaiveBayesClassifier.train(train_data)

In [28]:
nltk.classify.accuracy(classifier, test_data)

0.7751366120218579

In [29]:
data_modified=np.array([(" ".join(document), category)  for (document, category) in cleaned_data])

In [30]:
x_train=data_modified[:, 0][: limit_75]
x_test=data_modified[:, 0][limit_75: ]

y_train=data_modified[:, 1][: limit_75]
y_test=data_modified[:, 1][limit_75: ]

In [31]:
tfidf_vec=TfidfVectorizer(max_features=2000, min_df=5, max_df=0.15)

In [32]:
x_train_modified=tfidf_vec.fit_transform(x_train)

In [33]:
x_test_modified=tfidf_vec.transform(x_test)

In [34]:
classifier=SVC(kernel="linear")

In [35]:
classifier.fit(x_train_modified, y_train)

classifier.score(x_test_modified, y_test)

0.7885245901639344